In [29]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
import pickle

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 50)

## Step 1: Read in hold out data, scalers, and best model

In [30]:
#holdout set holds test features as oppose to test train 
holdout = pd.read_csv('data_for_final/kc_house_data_test_features.csv', index_col=0)
holdout.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [56]:
#read in the pickle file from Housing Prices of Kings County notebook into this jupyter notebook
pickle_in = open("data_for_final/model.pickle", "rb")
model = pickle.load(pickle_in)

In [57]:
#CSV of the columns from the best model 
selected_columns = pd.read_csv('data_for_final/selected_columns_bestmodel.csv')

In [58]:
selected_columns

,Unnamed: 0,0
0,0,bedrooms
1,1,bathrooms
2,2,sqft_living
3,3,floors
4,4,view
...,...,...
95,95,lat^2
96,96,lat long
97,97,lat waterfront_1
98,98,long waterfront_1


In [59]:
selected_columns.drop('Unnamed: 0', axis=1, inplace=True)

In [60]:
selected_columns

,0
0,bedrooms
1,bathrooms
2,sqft_living
3,floors
4,view
...,...
95,lat^2
96,lat long
97,lat waterfront_1
98,long waterfront_1


In [61]:
select_col = list(selected_columns['0'].values)

In [62]:
len(select_col)

100

## Step 2: Feature Engineering for holdout set

Perform the same transformations on our holdout data that were performed on the original data.

In [63]:
holdout

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,age,renovation_age
0,1974300020,4.0,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,53,0
1,1974300020,4.0,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918,53,0
2,3630020380,3.0,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576,15,0
3,1771000290,3.0,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565,44,0
4,5126310470,4.0,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916,15,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4318,263000018,3.0,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509,11,0
4319,6600060120,4.0,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200,6,0
4320,1523300141,2.0,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007,11,0
4321,291310100,3.0,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069,1410,1287,16,0


In [64]:
holdout.drop(columns=['date'], inplace = True)

KeyError: "['date'] not found in axis"

In [ ]:
holdout['bedrooms'] = holdout['bedrooms'].replace([0,33,11],[holdout['bedrooms'].median(),3, 4])
holdout['bedrooms']

In [ ]:
holdout['age'] = 2020 - holdout['yr_built']

In [ ]:
holdout['renovation_age'] = np.where(holdout['yr_renovated']>0, 2020 - holdout['yr_renovated'], 0)

In [ ]:
no_basement = holdout[holdout['sqft_basement'] == 0]
basement = holdout[holdout['sqft_basement'] != 0]

In [ ]:
waterfront = holdout[holdout['waterfront'] == 1]
no_water = holdout[holdout['waterfront'] == 0]

In [ ]:
df_dummy = pd.get_dummies(holdout, columns=['condition', 'waterfront'], drop_first='True')

In [ ]:
df_dummy.columns

In [ ]:
#columns with df_dummy to exclude from features
exclude = ['id', 'price', 'yr_built', 'yr_renovated', 'sqft_living15', 'sqft_lot15']

In [65]:
features = [x for x in df_dummy.columns if x not in exclude]

In [66]:
df_dummy.columns

Index(['id', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'view', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15',
       'age', 'renovation_age', 'condition_2', 'condition_3', 'condition_4',
       'condition_5', 'waterfront_1'],
      dtype='object')

In [67]:
poly = PolynomialFeatures(degree=2, include_bias=False)
poly_data = poly.fit_transform(df_dummy[features])
poly_data

array([[4.00e+00, 2.50e+00, 2.27e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [4.00e+00, 2.50e+00, 2.27e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [3.00e+00, 2.50e+00, 1.47e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       ...,
       [2.00e+00, 7.50e-01, 1.02e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [3.00e+00, 2.50e+00, 1.60e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00],
       [2.00e+00, 7.50e-01, 1.02e+03, ..., 0.00e+00, 0.00e+00, 0.00e+00]])

In [68]:
poly_data_columns = poly.get_feature_names(features)
holdout_poly = pd.DataFrame(poly_data, columns=poly_data_columns)

In [69]:
selected_columns

,0
0,bedrooms
1,bathrooms
2,sqft_living
3,floors
4,view
...,...
95,lat^2
96,lat long
97,lat waterfront_1
98,long waterfront_1


## Step 3: Predict the holdout set

In [70]:
y_pred = model.predict(holdout_poly[select_col])

In [71]:
y_pred

array([678444.76135349, 678444.76135349, 379972.28148174, ...,
       333616.92726517, 428888.49575329, 334423.84993839])

## Step 4: Export your predictions

In [72]:
pd.DataFrame(y_pred).to_csv('housing_pred_Raizel_Bernstein_3.csv')